# Init

In [1]:
import sys
import os
import json
import time
import re

from tqdm import tqdm

cwd = os.getcwd()
os.chdir(cwd)
sys.path.append('tools')

import chat
import parse_data
import sql

# Generator Answer

In [13]:
src_fname = 'stage_1-glm_4_plus-answer_generator-v1.0.0.json'
saved_fname = 'submit-stage_1-250125.json'
template_fname = 'submit_example.json'

src_fpath = os.path.join('answer_tmp', src_fname)
saved_fpath = os.path.join('answer', saved_fname)
template_fpath = os.path.join('data', template_fname)

In [14]:
raw_data = parse_data.read_json(src_fpath)
answers = parse_data.read_json(template_fpath)

for i in raw_data:
    tid = i['tid']
    try:
        answer = i['answer_generator']['stage_1']
        
        for j in answers:
            if j['tid'] == tid:
                j['team'][0]['answer'] = answer
    except:
        print(tid)

parse_data.write_json(answers, saved_fpath)

## Tokenizer

In [13]:
import hanlp

hanlp.pretrained.tok.ALL # 语种见名称最后一个字段或相应语料库
hanlp.pretrained.mtl.ALL
hanlp.pretrained.ner.ALL # 语种见名称最后一个字段或相应语料库

tok = hanlp.load(hanlp.pretrained.tok.COARSE_ELECTRA_SMALL_ZH)
# ner = hanlp.load(hanlp.pretrained.ner.MSRA_NER_ELECTRA_SMALL_ZH)

{'SIGHAN2005_PKU_CONVSEG': 'https://file.hankcs.com/hanlp/tok/sighan2005-pku-convseg_20200110_153722.zip',
 'SIGHAN2005_MSR_CONVSEG': 'https://file.hankcs.com/hanlp/tok/convseg-msr-nocrf-noembed_20200110_153524.zip',
 'CTB6_CONVSEG': 'https://file.hankcs.com/hanlp/tok/ctb6_convseg_nowe_nocrf_20200110_004046.zip',
 'PKU_NAME_MERGED_SIX_MONTHS_CONVSEG': 'https://file.hankcs.com/hanlp/tok/pku98_6m_conv_ngram_20200110_134736.zip',
 'LARGE_ALBERT_BASE': 'https://file.hankcs.com/hanlp/tok/large_corpus_cws_albert_base_20211228_160926.zip',
 'SIGHAN2005_PKU_BERT_BASE_ZH': 'https://file.hankcs.com/hanlp/tok/sighan2005_pku_bert_base_zh_20201231_141130.zip',
 'COARSE_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/tok/coarse_electra_small_20220616_012050.zip',
 'FINE_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/tok/fine_electra_small_20220615_231803.zip',
 'CTB9_TOK_ELECTRA_SMALL': 'https://file.hankcs.com/hanlp/tok/ctb9_electra_small_20220215_205427.zip',
 'CTB9_TOK_ELECTRA_BASE': 'http:/

{'OPEN_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/open_tok_pos_ner_srl_dep_sdp_con_electra_small_20201223_035557.zip',
 'OPEN_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH': 'https://file.hankcs.com/hanlp/mtl/open_tok_pos_ner_srl_dep_sdp_con_electra_base_20201223_201906.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_small_20210111_124159.zip',
 'CLOSE_TOK_POS_NER_SRL_UDEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_small_20220626_175100.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_base_20210111_124519.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ERNIE_GRAM_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_ernie_gram_base_aug_20210904_145403.zip',
 'KYOTO_EVAHAN_TOK_LEM_POS_UDEP_LZH': 'https

{'MSRA_NER_BERT_BASE_ZH': 'https://file.hankcs.com/hanlp/ner/ner_bert_base_msra_20211227_114712.zip',
 'MSRA_NER_ALBERT_BASE_ZH': 'https://file.hankcs.com/hanlp/ner/msra_ner_albert_base_20211228_173323.zip',
 'MSRA_NER_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/ner/msra_ner_electra_small_20220215_205503.zip',
 'CONLL03_NER_BERT_BASE_CASED_EN': 'https://file.hankcs.com/hanlp/ner/ner_conll03_bert_base_cased_en_20211227_121443.zip'}

In [15]:
tok.dict_force = {'JD.com, Inc.', '上海建工'}

print(tok(['600872的全称、A股简称、法人、法律顾问、会计师事务所及董秘是？', '今天是2021年12月24日，创近半年新高的股票有几只？', 'TOUR他是否已经退市了？（是或者否）', '铜陵有色金属集团股份有限公司的法人代表是谁？', '海信视像科技股份有限公司在什么时候成立的，XXXX-XX-XX？', '截止2021-06-17上海建工的近一周成交金额（万元）是多少？', 'JD.com, Inc.这家公司在美股英文名称是什么？']))

[['600872', '的', '全称', '、', 'A股', '简称', '、', '法人', '、', '法律顾问', '、', '会计师', '事务所', '及', '董秘', '是', '？'], ['今天', '是', '2021年12月24日', '，', '创', '近', '半年', '新高', '的', '股票', '有', '几只', '？'], ['TOUR', '他', '是否', '已经', '退市', '了', '？', '（', '是', '或者', '否', '）'], ['铜陵有色金属集团股份有限公司', '的', '法人', '代表', '是', '谁', '？'], ['海信视像科技股份有限公司', '在', '什么', '时候', '成立', '的', '，', 'XXXX-XX-XX', '？'], ['截止', '2021-06-17', '上海建工', '的', '近', '一周', '成交', '金额', '（', '万元', '）', '是', '多少', '？'], ['JD.com, Inc.', '这', '家', '公司', '在', '美股', '英文', '名称', '是', '什么', '？']]


In [2]:
print(tok(['600872的全称、A股简称、法人、法律顾问、会计师事务所及董秘是？', '今天是2021年12月24日，创近半年新高的股票有几只？', 'TOUR他是否已经退市了？（是或者否）', '铜陵有色金属集团股份有限公司的法人代表是谁？', '海信视像科技股份有限公司在什么时候成立的，XXXX-XX-XX？', '截止2021-06-17上海建工的近一周成交金额（万元）是多少？', 'JD.com, Inc.这家公司在美股英文名称是什么？'], tasks='ner*'))

[['600872', '的', '全称', '、', 'A股', '简称', '、', '法人', '、', '法律顾问', '、', '会计师', '事务所', '及', '董秘', '是', '？'], ['今天', '是', '2021年12月24日', '，', '创', '近', '半年', '新高', '的', '股票', '有', '几只', '？'], ['TOUR', '他', '是否', '已经', '退市', '了', '？', '（', '是', '或者', '否', '）'], ['铜陵有色金属集团股份有限公司', '的', '法人', '代表', '是', '谁', '？'], ['海信视像科技股份有限公司', '在', '什么', '时候', '成立', '的', '，', 'XXXX-XX-XX', '？'], ['截止', '2021-06-17', '上海', '建工', '的', '近', '一周', '成交', '金额', '（', '万元', '）', '是', '多少', '？'], ['JD.com', ',', 'Inc.', '这', '家', '公司', '在', '美股', '英文', '名称', '是', '什么', '？']]


In [17]:
tok.dict_force = {'JD.com, Inc.', '上海建工'}

print(tok(['600872的全称、A股简称、法人、法律顾问、会计师事务所及董秘是？', '今天是2021年12月24日，创近半年新高的股票有几只？', 'TOUR他是否已经退市了？（是或者否）', '铜陵有色金属集团股份有限公司的法人代表是谁？', '海信视像科技股份有限公司在什么时候成立的，XXXX-XX-XX？', '截止2021-06-17上海建工的近一周成交金额（万元）是多少？', 'JD.com, Inc.这家公司在美股英文名称是什么？']))

[['600872', '的', '全称', '、', 'A股', '简称', '、', '法人', '、', '法律顾问', '、', '会计师', '事务所', '及', '董秘', '是', '？'], ['今天', '是', '2021年12月24日', '，', '创', '近', '半年', '新高', '的', '股票', '有', '几只', '？'], ['TOUR', '他', '是否', '已经', '退市', '了', '？', '（', '是', '或者', '否', '）'], ['铜陵有色金属集团股份有限公司', '的', '法人', '代表', '是', '谁', '？'], ['海信视像科技股份有限公司', '在', '什么', '时候', '成立', '的', '，', 'XXXX-XX-XX', '？'], ['截止', '2021-06-17', '上海建工', '的', '近', '一周', '成交', '金额', '（', '万元', '）', '是', '多少', '？'], ['JD.com, Inc.', '这', '家', '公司', '在', '美股', '英文', '名称', '是', '什么', '？']]


In [7]:
import jieba

[word for word in jieba.cut('今天是2021年12月24日，创近半年新高的股票有几只？')]

['今天',
 '是',
 '2021',
 '年',
 '12',
 '月',
 '24',
 '日',
 '，',
 '创',
 '近半年',
 '新高',
 '的',
 '股票',
 '有',
 '几只',
 '？']